## Análisis de sentimientos con Text Analytics
By Fernanda Ochoa.

El presente notebook, desarrolla un ejemplo de como utilizar la data extraída por ThreadsAPI con la implementación de ```getData.py``` y el notebook de limpieza de datos ```threads.ipynb``` para trabajarlo con los servicios cognitivos de Azure.

[Azure Cognitive Services](https://learn.microsoft.com/es-MX/azure/cognitive-services/) es una plataforma basada en la nube que ofrece servicios de inteligencia artificial (IA). Estos servicios nos permiten integrar capacidades cognitivas en aplicaciones sin necesidad de tener conocimientos profundos de IA o ciencia de datos.


#### Categorías de servicios cognitivos

Los servicios cognitivos se pueden clasificar en cinco áreas principales:

+ [Voz](https://learn.microsoft.com/es-MX/azure/cognitive-services/): 
  + Voz a texto
  + Texto a voz
  + Traducción de voz
  + Reconocimiento de voz
+ [Lenguaje](https://learn.microsoft.com/es-MX/azure/cognitive-services/):
  + Reconocimiento de entidades
  + Análisis de opiniones
  + Respuestas a preguntas
  + Reconocimiento de lenguaje conversacional
  + Traductor
+ [Visión](https://learn.microsoft.com/es-MX/azure/cognitive-services/):
  + Análisis de Imágenes y Video
  + Modelos personalizados
+ [Decisión](https://learn.microsoft.com/es-MX/azure/cognitive-services/):
  + Detector de anomalías
  + Moderador de contenido
  + Personalizer
+ [Servicio Azure OpenAI](https://learn.microsoft.com/es-MX/azure/cognitive-services/openai/concepts/models):
  + Dall E
  + GPT-4
  + GPT-3.5
  + Incrustaciones

#### Herramientas

* Pandas
    * Es una biblioteca considerada una extensión de Numpy para la manipulación y análisis de datos con Python.
    * [Pandas Docs](https://pandas.pydata.org/)
* Azure AI TextAnalytics:
  * Es un servicio basado en la nube que proporciona funciones de procesamiento de lenguaje natural (NLP) para comprender y analizar texto.
  * Es necesaria una suscripción de Azure para obtener las credenciales como ```APIKey``` y ```Url Endpoint```.
  * [Azure AI TextAnalytics Docs](https://pypi.org/project/azure-ai-textanalytics/)
* Dot env:
  * Biblioteca para leer los pares clave-valor de un archivo .env y  establecerlos como variables de entorno. Ayuda en el desarrollo de aplicaciones siguiendo los principios de 12 factores.
  * [Dot env Docs](https://pypi.org/project/python-dotenv/)

#### Requisitos
* Dataset: Conjunto de datos en csv con el que vamos a trabajar.
  * Puedes encontrar un ejemplo en la carpeta: ```data``` en formato ```csv```.
* Suscripción de Azure: Ingresar al [portal de azure](https://portal.azure.com), crear una suscripción, agregar una tarjeta (usamos la tarifa gratuita por lo que no hay costos) y listo.
* [Activar suscripción](https://azure.microsoft.com/es-mx/free/)
* Credenciales text analytics: 
  * Ingresar al [portal de azure](https://portal.azure.com)
  * [Seguir el tutorial](https://learn.microsoft.com/es-mx/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Canomaly-detector%2Clanguage-service%2Ccomputer-vision%2Cwindows)

#### Instalamos las Bibliotecas

* [Text analytics](https://pypi.org/project/azure-ai-textanalytics/):
  * Visual Studio Code: ```%pip install azure-ai-textanalytics```
  * Google Colab: ```!pip install azure-ai-textanalytics```
* [Dot env](https://pypi.org/project/python-dotenv/): 
  * Visual Studio Code: ```%pip install python-dotenv```
  * Google Colab: ```!pip install python-dotenv```

#### Credenciales

Una vez creados los servicios cognitivos en el [portal de azure](https://portal.azure.com). Se debe crear en la raíz del proyecto un archivo llamado ```.env```, este archivo de configuración, contiene las variables de entorno del proyecto, es decir las credenciales de acceso al API de los servicios cognitivos.

``` Contenido de mi archivo .env
API_KEY = 'Reemplazar por tus credenciales'
API_ENDPOINT = 'Reemplazar por tus credenciales'
API_REGION = 'Reemplazar por tus credenciales'
```

Recuerda crear el archivo: ```.gitignore``` y añadir tu archivo ```.env``` si planeas subirlo a un repositorio de git.

``` Contenigo de mi archivo .gitignore
# Ignorar archivo .env que puede contener información sensible
.env
```
---

#### Procedimiento

In [1]:
# Instalamos requests
%pip install --upgrade requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Instalamos azure-ai-textanalytics
%pip install azure-ai-textanalytics

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Instalamos python-dotenv
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


#### Importamos las bibliotecas y variables de entorno

In [4]:
# Para importar las variables de entorno
from dotenv import load_dotenv
load_dotenv()
import os

In [5]:
# Importamos las bibliotecas de Azure: Credentials y TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

In [6]:
# Importamos la biblioteca pandas
import pandas as pd

In [7]:
# Definimos las credenciales del API
endpoint = os.getenv('API_ENDPOINT')
apiKey = os.getenv('API_KEY')

In [8]:
# Autenticamos el cliente
client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(apiKey))

In [9]:
# Importamos nuestro csv con la data
data = pd.read_csv('../data/cleanData.csv')

In [10]:
# Estructuramos el dataframe con las columnas de interés: Text y Likes
data.columns = ['Thread', 'Likes']

In [11]:
# Obtenemos la longitud de cada texto
data['len'] = data['Thread'].apply(len)

In [12]:
# Cambiamos la posición de la columna len al lado de la columna de texto del thread, obteniendo el nombre de la columna en la posición 2
columnToMove = data.columns[2]

col = data.pop(columnToMove)

data.insert(1, columnToMove, col)

In [13]:
# Comprobamos el cambio de lugar de columnas
data.head(2)

,Thread,len,Likes
0,Amanecimos con buenas noticias 💪🏽Ya podemos es...,141,14
1,"Listo, ya solo documento algunos ejemplos con ...",231,33


In [14]:
# Convertimos el dataframe a una lista 
textThreads = data["Thread"].tolist()

In [15]:
# Crear una lista para guardar los sentimientos de las revisiones
sentiments = []

In [16]:
# Procesamos el texto por lotes de 10 (límite de la API)
for i in range(0, len(textThreads), 10):
    batch = textThreads[i : i + 10]
    result = client.analyze_sentiment(batch, show_opinion_mining=True)
    docs = [doc for doc in result if not doc.is_error]

    for idx, doc in enumerate(docs):
        sentiments.append(doc.sentiment)


In [17]:
# Si la cantidad de threads no son un múltiplos de 10, es posible que la longitud de
# 'sentiments' sea menor que la longitud de 'threads'. En este caso, agregamos como sentimiento
# desconocido para los threads restantes.
while len(sentiments) < len(textThreads):
    sentiments.append("desconocido")

In [18]:
# Agregamos la columna Sentiment
data['Sentiment'] = sentiments

In [19]:
# Mostramos el dataframe con la columna Sentiment
data.head()

,Thread,len,Likes,Sentiment
0,Amanecimos con buenas noticias 💪🏽Ya podemos es...,141,14,positive
1,"Listo, ya solo documento algunos ejemplos con ...",231,33,neutral
2,Ya les ando preparando el repo para que puedan...,183,11,neutral
3,"¿Cómo va su sábado?, ¿Ya tomaron café?",38,15,neutral
4,Si llego a mil seguidores el fin les explico c...,215,31,neutral


Si así lo deseamos, podemos exportarlo a un csv, complementarlo con más api's, etc.

Threads API: Análisis de Sentimientos con Azure y Threads API

Material desarrollado por: Fernanda Ochoa.

Contacto:
* GitHub: [FernandaOchoa](https://github.com/FernandaOchoa)
* Twitter: [@imonsh](https://twitter.com/imonsh)
* Instagram: [fherz8a](https://instagram.com/fherz8a)
* Linkedin: [Fernanda Ochoa](https://www.linkedin.com/in/fernandaochoa8/)